# Disease Symptom Resolution

By Paul Gaudin

## 1. Querying for Disease Signs and Symptoms

Disease symptoms returned via the HPO endpoint are not explanatory, as they are formatted as HPIDs, are not specific to "Phenotypic aspects," must be queried again to retreive frequency of the phenotype, nd do not include all the synonyms for a diesase. The following funciton returns a dictionary formatted as {PhenotypeId: {names: [str, str], frequencey: str}, ...} 

In [1]:
import requests 
import pandas as pd
import math
from biothings_explorer.user_query_dispatcher import FindConnection
from biothings_explorer.hint import Hint
ht = Hint()

In [35]:
def get_disease_symptoms(disease_name):
    request_url = 'http://mydisease.info/v1/query?q=hpo.disease_name:"' + disease_name + '"&fields=hpo'
    print("request url: " + request_url)
    r = requests.get(request_url)
    res = r.json()
    # print(res)
    result_number = 0
    disease_info = res['hits'][result_number]
    print("disease symptoms for:")
    print(disease_info['hpo']['disease_name'])
    hp_symptom_dict = {}
    for x in disease_info['hpo']['phenotype_related_to_disease']:
        if(x["aspect"].lower() == 'p'):
            try:
                r1 = requests.get('https://biothings.ncats.io/hpo/phenotype/' + x['frequency'])
                res1 = r1.json()
                frequency = True
            except:
                frequency = False; 
            r = requests.get('https://biothings.ncats.io/hpo/phenotype/' + x['hpo_id'])
            res = r.json()
            if(('_id' in res) & ('name' in res)):
                hp_symptom_dict[res['_id']] = {
                    'names' : [res['name'].lower()],
                    'frequency' : res1['name'] if frequency else 'Unknown'
                }
            if('synonym' in res):
                if('exact' in res['synonym']):
                    for name in res['synonym']['exact']:
                        if name.lower() not in hp_symptom_dict[res['_id']]['names']: 
                            hp_symptom_dict[res['_id']]['names'].append(name.lower())
    return(hp_symptom_dict)


In [36]:
xp_symptom_dict = get_disease_symptoms('xeroderma pigmentosum')
# xp_symptom_dict

request url: http://mydisease.info/v1/query?q=hpo.disease_name:"xeroderma pigmentosum"&fields=hpo
disease symptoms for:
Xeroderma pigmentosum


In [37]:
pd.DataFrame.from_dict(xp_symptom_dict, orient='index')

,names,frequency
HP:0000028,"[cryptorchidism, cryptorchism, undescended tes...",Frequent
HP:0000135,"[hypogonadism, decreased activity of gonads]",Very frequent
HP:0000164,"[abnormality of the dentition, abnormal dentit...",Very frequent
HP:0000252,"[microcephaly, abnormally small cranium, abnor...",Occasional
HP:0000365,"[hearing impairment, congenital deafness, cong...",Occasional
HP:0000407,"[sensorineural hearing impairment, hearing los...",Frequent
HP:0000486,"[strabismus, cross-eyed, squint, squint eyes]",Frequent
HP:0000491,"[keratitis, corneal inflammation]",Frequent
HP:0000498,"[blepharitis, cellulitis of eyelids, inflammat...",Occasional
HP:0000518,"[cataract, cataracts, clouding of the lens of ...",Frequent


### 1.1 Symptom specificity 

By looking at the number of nodes a specific phenotype is connected to, we can get a relative measure of how prevalent a phenotype is and how specific it may be to a disease. Thereafter, by looking at specificity and frequency of a sign or symptom, we can get a measure of how "characteristic" a sign or symptom is of a disease, which is reflected in the Individual Symptom Scores (ISS) in the following dataframe outputs. 

In [39]:
def get_symtpom_prevalence(hp_symptom_dict, disease_name, node_intermediates):
    # go through all symptoms in dict - key is the hpid
    for key in hp_symptom_dict:
        print(str(key) + ' is running')
        edges_out_count = 0
        UMLS = ''
        # a sign or symptom may show up as a phenotypic feature, a disease, or biological process
        for y in ['PhenotypicFeature','Disease','BiologicalProcess']:
            if y == 'PhenotypicFeature':
                a = ht.query(key)[y]
                if len(a) > 0: 
                    b = a[0]
                    if 'UMLS' in b: 
                        UMLS = b['UMLS']
                    for node_intermediate in node_intermediates:
                        try: 
                            fc = FindConnection(input_obj=b, output_obj=node_intermediate, intermediate_nodes=None)
                            fc.connect(verbose=False)
                            df = fc.display_table_view()
                            if(df.shape[0] > 0):
                                df = df[df["output_name"] != disease_name]
                                edges_out_count = edges_out_count + df.shape[0]
                        except: 
                             print(str(y) + " input to " + node_intermediate + " output for " + str(key) + ' failed')
            if(y =='Disease') | (y == 'BiologicalProcess'):
                for z in hp_symptom_dict[key]["names"]:
                    if((y == 'Disease') & (len(UMLS) > 0)): 
                        try: 
                            a = ht.query(UMLS)[y]
                        except: 
                            a = []
                    else:
                        try: 
                            a = ht.query(z)[y]
                        except: 
                            a = []
                    for b in a: 
                        if b['name'].lower() == z.lower():
                            for node_intermediate in node_intermediates:
                                try: 
                                    fc = FindConnection(input_obj=b, output_obj=node_intermediate, intermediate_nodes=None)
                                    fc.connect(verbose=False)
                                    df = fc.display_table_view()
                                    if(df.shape[0] > 0):
                                        df = df[df["output_name"] != disease_name]
                                        edges_out_count = edges_out_count + df.shape[0]

                                except: 
                                    print(str(y) + " input to " + node_intermediate + " output for " + str(key) + ' failed')
        hp_symptom_dict[key]["edges_out_count"] = edges_out_count
    return(hp_symptom_dict)

In [1]:
node_type_list = (['Gene', 'SequenceVariant', 'ChemicalSubstance', 'Disease', 
                'MolecularActivity', 'BiologicalProcess', 'CellularComponent', 
                'Pathway', 'AnatomicalEntity', 'PhenotypicFeature'])



xp_symptom_dict = get_symtpom_prevalence(xp_symptom_dict, 'xeroderma pigmentosum', node_type_list) 


In [41]:
def format_disease_symptom_table(disease_symptom_dict):
    disease_symptom_df = pd.DataFrame.from_dict(disease_symptom_dict, orient='index').sort_values(by=['edges_out_count'])
    disease_symptom_df = pd.concat([disease_symptom_df[disease_symptom_df["frequency"] == "Very frequent"],
               disease_symptom_df[disease_symptom_df["frequency"] == "Frequent"],
               disease_symptom_df[disease_symptom_df["frequency"] == "Occasional"],
               disease_symptom_df[disease_symptom_df["frequency"] == "Rare"],
               disease_symptom_df[disease_symptom_df["frequency"] == "Unknown"]
              ])
    individual_symptom_scores = [];
    for index,x in disease_symptom_df.iterrows():
        if(x["frequency"] == "Very frequent"):
            score = 20 / math.sqrt(int(x["edges_out_count"]))
        elif(x["frequency"] == "Frequent"):
            score = 15 / math.sqrt(int(x["edges_out_count"]))
        elif(x["frequency"] == "Occasional"):
            score = 10 / math.sqrt(int(x["edges_out_count"]))
        elif(x["frequency"] == "Rare"):
            score = 5 / math.sqrt(int(x["edges_out_count"]))
        elif(x["frequency"] == "Unknown"):
            score = 5 / math.sqrt(int(x["edges_out_count"]))
        individual_symptom_scores.append(score)

    # print(len(individual_symptom_scores))
    disease_symptom_df["ISS"] = individual_symptom_scores
    
    return(disease_symptom_df)

In [42]:
xp_symptoms_df = format_disease_symptom_table(xp_symptom_dict)
xp_symptoms_df

,names,frequency,edges_out_count,ISS
HP:0000524,"[conjunctival telangiectasia, conjunctival tel...",Very frequent,52,2.773501
HP:0001029,[poikiloderma],Very frequent,59,2.603778
HP:0001480,[freckling],Very frequent,78,2.264554
HP:0100585,"[telangiectasia of the skin, teleangiectasia o...",Very frequent,103,1.970659
HP:0001009,"[telangiectasia, cutaneous telangiectasia, tel...",Very frequent,136,1.714986
HP:0001072,"[thickened skin, pachydermia, thick skin]",Very frequent,166,1.552301
HP:0000992,"[cutaneous photosensitivity, photosensitive sk...",Very frequent,222,1.342312
HP:0006887,"[intellectual disability, progressive, mental ...",Very frequent,235,1.304656
HP:0000963,[thin skin],Very frequent,421,0.974740
HP:0100543,"[cognitive impairment, abnormality of cognitio...",Very frequent,521,0.876216


In [9]:
disease = ht.query('xeroderma pigmentosum')['Disease'][0]

In [10]:
fc = FindConnection(input_obj=disease, output_obj='PhenotypicFeature', intermediate_nodes=None)
fc.connect(verbose=False)
df = fc.display_table_view()

In [11]:
df

,input,input_type,pred1,pred1_source,pred1_api,pred1_pubmed,output_type,output_name,output_id
0,XERODERMA PIGMENTOSUM,Disease,related_to,None,BioLink API,None,PhenotypicFeature,"FLAT, DISCOLORED AREA OF SKIN",UMLS:C0332573
1,XERODERMA PIGMENTOSUM,Disease,related_to,None,BioLink API,None,PhenotypicFeature,CRANIOFACIAL HYPEROSTOSIS,UMLS:C4280516
2,XERODERMA PIGMENTOSUM,Disease,related_to,None,BioLink API,None,PhenotypicFeature,ATROPHIC SKIN,UMLS:C0151514
3,XERODERMA PIGMENTOSUM,Disease,related_to,None,BioLink API,None,PhenotypicFeature,DECREASED BODY HEIGHT,UMLS:C0349588
4,XERODERMA PIGMENTOSUM,Disease,related_to,None,BioLink API,None,PhenotypicFeature,NEUROPATHY,UMLS:C0031117
5,XERODERMA PIGMENTOSUM,Disease,related_to,None,BioLink API,None,PhenotypicFeature,ADHESION OF EYELIDS,UMLS:C0339182
6,XERODERMA PIGMENTOSUM,Disease,related_to,None,BioLink API,None,PhenotypicFeature,CANCER OF SKIN PIGMENT CELLS,UMLS:C0025202
7,XERODERMA PIGMENTOSUM,Disease,related_to,None,BioLink API,None,PhenotypicFeature,ARTHRALGIA,UMLS:C0003862
8,XERODERMA PIGMENTOSUM,Disease,related_to,None,BioLink API,None,PhenotypicFeature,DELAYED BONE AGE,UMLS:C0541764
9,XERODERMA PIGMENTOSUM,Disease,related_to,None,BioLink API,None,PhenotypicFeature,ABNORMAL TISSUE MASS,UMLS:C0027651


In [12]:
disease

{'MONDO': 'MONDO:0019600',
 'DOID': 'DOID:0050427',
 'UMLS': 'C0043346',
 'name': 'xeroderma pigmentosum',
 'MESH': 'D014983',
 'ORPHANET': '910',
 'primary': {'identifier': 'MONDO',
  'cls': 'Disease',
  'value': 'MONDO:0019600'},
 'display': 'MONDO(MONDO:0019600) DOID(DOID:0050427) ORPHANET(910) UMLS(C0043346) MESH(D014983) name(xeroderma pigmentosum)',
 'type': 'Disease'}

In [15]:
ht.query('HP:0003355')

{'Gene': [],
 'SequenceVariant': [],
 'ChemicalSubstance': [],
 'Disease': [],
 'PhenotypicFeature': [{'UMLS': 'C0238621',
   'HP': 'HP:0003355',
   'name': 'Aminoaciduria',
   'primary': {'identifier': 'UMLS',
    'cls': 'PhenotypicFeature',
    'value': 'C0238621'},
   'display': 'UMLS(C0238621) HP(HP:0003355) name(Aminoaciduria)',
   'type': 'PhenotypicFeature'}],
 'MolecularActivity': [],
 'BiologicalProcess': [],
 'CellularComponent': [],
 'Pathway': [],
 'AnatomicalEntity': [],
 'Cell': []}